In [1]:
%matplotlib inline


In [2]:
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

import matplotlib
matplotlib.axes.Axes.bar
matplotlib.pyplot.bar
matplotlib.projections.polar

<module 'matplotlib.projections.polar' from '/anaconda3/envs/PythonData/lib/python3.6/site-packages/matplotlib/projections/polar.py'>

In [3]:
import numpy as np
import pandas as pd

In [4]:
import datetime as dt

In [5]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [6]:
database_path = "../Resources/hawaii.sqlite"

In [7]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite", echo=False)
conn = engine.connect()

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
# DETERMINE TABLE NAMES

inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# DETERMINE COLUMN NAMES IN MEASUREMENT

columns = inspector.get_columns('Measurement')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# DETERMINE COLUMN NAMES IN STATION

columns = inspector.get_columns('Station')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
data = pd.read_sql("SELECT * FROM Measurement", conn)
data.head()

In [ ]:
station_data = pd.read_sql("SELECT * FROM Station", conn)
station_data.head()

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# LAST DATE IN TABLE

last_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
print(last_date)

In [ ]:
# FIND DATE STARTING 12 MONTHS BACK

one_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print(one_year)

In [ ]:
# RUN QUERY ON PRECIP

precip = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date > one_year).\
    order_by(Measurement.date).all()

In [ ]:
# SEND TO DATAFRAME FOR VIEWING

precip_df = pd.DataFrame(precip)

precip_df.head()

In [ ]:
numpy_array = precip_df.values
numpy_array

In [ ]:
# SET MONTH AS INDEX COLUMN

precip_df.set_index("date").head()

In [ ]:
# SORT DATAFRAME BY DATE (hmmm...was already sorted by date)

precip_df.sort_values(by='date').head()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
# Set x axis and tick locations

precip_df.plot('date', 'prcp', color="g", alpha=0.25)

plt.xticks(rotation=45)

plt.tight_layout()

plt.title("Precipitation over 12 Months")
plt.xlabel("Date")
plt.ylabel("Precipitation")

plt.show()

In [ ]:
# (IGNORE THIS) JUST HAVING SOME FUN HERE EXPERIMENTING WITH A RADIUS BAR GRAPH

# Compute pie slices

sample = np.random.uniform(low=0.5, high=10.0, size=(12,))
N = 365
theta = np.linspace(0, 2 * np.pi, N, endpoint=False)
height = numpy_array
radii = 8 * np.random.rand(N)
width = np.pi / 200 * np.random.rand(N)
colors = plt.cm.viridis(radii / 10.)

ax = plt.subplot(111, projection='polar')
ax.bar(theta, radii, width=width, bottom=0.0, color=colors, alpha=0.75)

ax.set_xticks(theta)
ax.set_xticklabels({'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC'})
ax.yaxis.grid(False)

plt.show()

In [ ]:
#import matplotlib.pyplot as plt
#import numpy as np

#N = 365

#theta = precip_df

#ax = plt.subplot(111, polar=True)

#bars = ax.bar(theta, height,
              #colors = plt.cm.viridis(radii / 10.),
              #bottom=0.0,
              #alpha=0.5)

#ax.set_xticks(theta)
#ax.set_xticklabels(range(1, len(theta)+1))
#ax.yaxis.grid(True)

#plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

precip_df.describe()

In [ ]:
# Design a query to show how many stations are available in this dataset?

station_count = session.query(Station).count()
print(station_count)

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


# TOTAL STATION ACTIVITY
station_activity = session.query(Measurement.tobs).count()
print(station_activity)


In [ ]:
# STATION ACTIVITY IN DESCENDING ORDER

station_obs = session.query(Measurement.station, func.count(Measurement.tobs)).group_by(Measurement.station).\
               order_by(func.count(Measurement.tobs).desc()).all()
print(station_obs)

In [ ]:
# STATION ACTIVITY IN DESCENDING ORDER AS DATAFRAME

obs_df = pd.DataFrame(station_obs, columns=['Station', 'tobs'])

obs_df.head(10)

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?

temperature = session.query(Measurement.tobs).\
               order_by(Measurement.tobs).all()

print(temperature)

In [ ]:
temp_df = pd.DataFrame(temperature)
print(temp_df)

In [ ]:
# MINIMUM TEMPERATURE

min_temp = temp_df['tobs'].min()
print(min_temp)

In [ ]:
# MAXIMUM TEMPERATURE

max_temp = temp_df['tobs'].max()
print(max_temp)

In [ ]:
# AVERAGE TEMPERATURE

avg_temp = temp_df['tobs'].mean()
print(avg_temp)

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

high_station = session.query(Measurement.station, Measurement.date, Measurement.tobs).\
    filter(Measurement.station == 'USC00519281').\
    filter(Measurement.date > one_year).\
    order_by(Measurement.date).all()

In [ ]:
plt.hist(temp_df['tobs'],12)
plt.xlabel("Temperature")
plt.ylabel("Observations")
plt.title("High Station Analysis")
plt.show()

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.

def calc_temps(start_date, end_date):
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()
temp_range = (calc_temps('2017-08-23', '2017and-08-30'))
print(temp_range)

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)

fig, ax = plt.subplots()
ax.boxplot(temp_range, patch_artist=True)
ax.set_title('Trip Avg Temp')
ax.set_ylabel("Temperature")
ax.set_xlabel("Trip")
plt.show()

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation

# JOIN TABLES FOR CATEGORIES NEEDED

sel = [Measurement.prcp, Measurement.station, Station.name, Station.latitude, Station.longitude, Station.elevation]
precip_per_station = session.query(*sel).all()

In [ ]:
columns = inspector.get_columns('sel')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
final_rain_df = pd.DataFrame(precip_per_station)

final_rain_df.head()

In [ ]:
# DATAFRAME WITH EVERYTHING DESCENDING BUT NOT SUMMED

final_rain_df.sort_values(by='prcp', ascending=False)

In [ ]:
rain_totals = final_rain_df.groupby("station").count()["prcp"].sort_values(ascending = False)
rain_totals

In [ ]:
rain_totals_df = pd.DataFrame(rain_totals)
rain_totals_df

In [ ]:
session.query(Station.station, 
              func.sum(Measurement.prcp), 
              Station.name, 
              Station.latitude, 
              Station.longitude, 
              Station.elevation).\
    group_by(Station.station).\
    order_by(func.sum(Measurement.prcp).desc()).all()

In [ ]:
# FLASK API COMPONENT STARTS HERE

In [ ]:
# IMPORT FLASK + JASONIFY

from flask import Flask, jsonify

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route("/")
def home():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start> and /api/v1.0/<start>/<end><br/>"
    )








@app.route("/api/v1.0/passengers")
def passengers():
    """Return a list of passenger data including the name, age, and sex of each passenger"""
    # Query all passengers
    session = Session(engine)
    results = session.query(Passenger.name, Passenger.age, Passenger.sex).all()

    # Create a dictionary from the row data and append to a list of all_passengers
    all_passengers = []
    for name, age, sex in results:
        passenger_dict = {}
        passenger_dict["name"] = name
        passenger_dict["age"] = age
        passenger_dict["sex"] = sex
        all_passengers.append(passenger_dict)

    return jsonify(all_passengers)


In [ ]:
# PRECIPITATION

@app.route("\/api/v1.0/precipitation/<date>")
def precipitation(date):
    
    canonicalized = date.replace(" ", "").lower()
    for prcp in Measurement:
        search_term = Measurement["date"].replace(" ", "").lower()

        if search_term == canonicalized:
            return jsonify(prcp)

    return jsonify({"error": f"Weather for {date} not found."}), 404

In [ ]:
# STATIONS

@app.route("/api/v1.0/stations")
def stations():
    session = Session(engine)
    results = session.query(Station.station).all()

    # Convert list of tuples into normal list
    all_stations = list(np.ravel(results))

    return jsonify(all_stations)

In [ ]:
# TOBS

# query for the dates and temperature observations from a year from the last data point.

In [ ]:
# FIND LAST DATE

session.query(Measurement.tobs).order_by(Measurement.date.desc()).first()

In [ ]:
# QUERY LAST YEAR'S DATA

query_last_year = Measurement.date(2011, 4, 8) - Measurement.timedelta(days=365)
print("Query Last Year Temperatures: ", query_last_year)

In [ ]:
# START DATE END DATE



In [ ]:
if __name__ == "__main__":
    app.run(debug=True)